<a href="https://colab.research.google.com/github/eliwagnercode/SmokeyBear/blob/main/CaliforniaWildfireDashboardApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package Installation

In [ ]:
!pip install geopandas
!pip install earthengine-api
!pip install geemap
!pip install dash

# Raw Data Acquisition

In [ ]:
# @title # Download archive from US Forest Service
!wget https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.6/RDS-2013-0009.6_SQLITE.zip

# Unarchive data
!unzip /content/RDS-2013-0009.6_SQLITE.zip

In [ ]:
# @title # Extract data from SQLite database

import sqlite3
conn = sqlite3.connect('/content/Data/FPA_FOD_20221014.sqlite')
cur = conn.cursor()
cur.execute("PRAGMA table_info('Fires')")
sql_cols = cur.fetchall()
df_cols = []
for col in sql_cols:
  df_cols.append(col[1])
cur.execute("select * from 'Fires'")
sql_rows = cur.fetchall()
df_rows = []
for row in sql_rows:
  df_rows.append(list(row))

# Convert to Pandas DataFrame
import pandas as pd
gdf_USFires = pd.DataFrame(data=df_rows,columns=df_cols)

# Drop columns and improve readability
gdf_USFires = gdf_USFires[
    ['FPA_ID','LONGITUDE','LATITUDE','FIRE_SIZE','FIRE_SIZE_CLASS',
     'FIRE_YEAR','DISCOVERY_DATE','STATE']
    ].rename(
    columns={
    'LONGITUDE':'lon',
    'LATITUDE':'lat',
    'FIRE_SIZE':'fireSize',
    'FIRE_SIZE_CLASS':'fireClass',
    'FIRE_YEAR':'fireYear',
    'DISCOVERY_DATE':'fireDate',
    'STATE':'stateName'}
    ).drop_duplicates('FPA_ID')
gdf_USFires

In [ ]:
# @title # Convert DataFrame to GeoDataFrame

import geopandas as gpd # !pip install geopandas

gdf_USFires = gpd.GeoDataFrame(gdf_USFires,
    geometry = gpd.points_from_xy(
        gdf_USFires['lon'], gdf_USFires['lat'])
    ).set_crs(epsg=4326
        )[['geometry','FPA_ID','fireSize','fireClass','fireYear','fireDate','stateName']]

In [ ]:
# @title # Download US county geometries from Google Earth Engine API

# Import, authenticate, and initialize Google Earth Engine;
import ee # !pip install earthengine-api
ee.Authenticate()
ee.Initialize()
import geemap # !pip install geemap
import json

gdf_USCounties = geemap.ee_to_geopandas(
    ee.FeatureCollection("TIGER/2016/Counties")
).set_crs(epsg=4326)
gdf_USCounties = gdf_USCounties.rename(
    columns={'GEOID':'countyFIPS',
             'STATEFP':'stateFIPS',
             'NAMELSAD':'countyName'})
gdf_USCounties = gdf_USCounties[['geometry','stateFIPS','countyFIPS','countyName']]

In [ ]:
# @title # Assign wildfire ignition points to US counties
gdf_USFires = gpd.sjoin(
    gdf_USFires,gdf_USCounties,
    how='left'
    ).drop(columns=['index_right'])
gdf_USFires

In [ ]:
# @title # Create subsets for California counties
gdf_CACounties = gdf_USCounties.copy()[gdf_USCounties['stateFIPS']=='06']
geojson_CACounties = json.loads(gdf_CACounties[['geometry']].to_json())

gdf_CAFires = gdf_USFires.copy()
gdf_CAFires = gdf_CAFires[
    gdf_CAFires['stateFIPS']=='06']
gdf_CAFires

In [ ]:
# @title # Calculate total burn area by county and year
df_USFires_BurnAreaByCountyAndYear = gdf_USFires.groupby(
    by=['fireYear','countyFIPS','countyName']
    )['fireSize'].sum().to_frame().reset_index().rename(
    columns={
        'fireYear':'Year',
        'countyFIPS':'FIPS',
        'countyName':'County Name',
        'fireSize':'Total Burn Area'}
)

df_CAFires_BurnAreaByCountyAndYear = gdf_CAFires.groupby(
    by=['fireYear','countyFIPS','countyName']
    )['fireSize'].sum().to_frame().reset_index().rename(
    columns={
        'fireYear':'Year',
        'countyFIPS':'FIPS',
        'countyName':'County Name',
        'fireSize':'Total Burn Area'}
)

In [ ]:
# @title # Download clean data for easier loading
from google.colab import files

def download_df_as_csv(df,filename):
  with open(filename, 'w', encoding = 'utf-8-sig') as f:
    df.to_csv(f)
  files.download(filename)

filename = 'USFires_BurnAreaByCountyAndYear.csv'
download_df_as_csv(df_USFires_BurnAreaByCountyAndYear,filename)

filename = 'CAFires_BurnAreaByCountyAndYear.csv'
download_df_as_csv(df_CAFires_BurnAreaByCountyAndYear,filename)

filename = 'gdf_USCounties.csv'
download_df_as_csv(gdf_USCounties,filename)

filename = 'gdf_CACounties.csv'
download_df_as_csv(gdf_CACounties,filename)

# Choropleth Map Dashboard

In [2]:
# @title # Load GeoDataFrames from CSV
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json

def csv_to_gdf(filepath):
  gdf = pd.read_csv(filepath)
  gdf = gdf.drop(gdf.columns[0],axis=1)
  gdf['geometry'] = gdf['geometry'].apply(wkt.loads) # Convert geometry string to WKT geometry object
  gdf = gpd.GeoDataFrame(gdf,geometry='geometry').set_crs(epsg=4326) # Convert df to gdf
  return gdf

df_CAFires_BurnAreaByCountyAndYear = pd.read_csv(
    'https://raw.githubusercontent.com/eliwagnercode/SmokeyBear/main/CAFires_BurnAreaByCountyAndYear.csv')

gdf_CACounties = csv_to_gdf(
    'https://raw.githubusercontent.com/eliwagnercode/SmokeyBear/main/gdf_CACounties.csv')
gdf_CACounties.index = gdf_CACounties['countyFIPS']
geojson_CACounties = json.loads(gdf_CACounties[['geometry']].to_json())

In [15]:
# @title # Build Choropleth Map Dashboard with Plotly
import plotly.express as px

df = df_CAFires_BurnAreaByCountyAndYear.copy()
Q3 = df['Total Burn Area'].quantile(0.75)
Q1 = df['Total Burn Area'].quantile(0.25)
IQR = Q3 - Q1
range_color_max = Q3 + (1.5*IQR)
fig = px.choropleth_mapbox(
    data_frame = df,
    geojson=geojson_CACounties,
    locations='FIPS',
    hover_name='County Name',
    color='Total Burn Area',
    color_continuous_scale='blues',
    range_color=(0,range_color_max),
    mapbox_style='carto-positron',
    zoom=3,
    center={'lat': 37.0902, 'lon': -119.4179},
    opacity=0.8,
    animation_frame = 'Year',
    animation_group = 'FIPS',
    title = 'Total Burn Area By County And Year'
)
fig.show()

In [25]:
# @title # Launch Dashboard as Dash app

# Create a Dash app
from dash import Dash, dcc, html, Input, Output # !pip install dash
import plotly.express as px

app = Dash(__name__)

# Select data
df = df_CAFires_BurnAreaByCountyAndYear.copy()
range_color_max = df['Total Burn Area'].quantile(0.75)
# Define the layout of the app
app.layout = html.Div([
    dcc.Slider(
        id='year-slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].min(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ),
    dcc.Graph(id='choropleth')
])

# Define the callback function
@app.callback(
    Output('choropleth', 'figure'),
    [Input('year-slider', 'value')]
)
def update_choropleth(selected_year):
    filtered_data = df[df['Year'] == selected_year]
    fig = px.choropleth_mapbox(
        data_frame=filtered_data,
        geojson=geojson_CACounties,
        locations='FIPS',
        hover_name = 'County Name',
        color='Total Burn Area',
        color_continuous_scale='blues',
        range_color=(0,range_color_max),
        mapbox_style='carto-positron',
        zoom=5.5,
        center={'lat': 37.0902, 'lon': -119.4179},
        opacity=0.8,
        title = 'Total Burn Area By County And Year'
    )
    fig.update_layout(margin={'r': 10, 't': 50, 'l': 10, 'b': 10})
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>